In [2]:
%load_ext autoreload
%autoreload 2

In [34]:
import os
from pathlib import Path
from vitssm.data import get_dataset
from omegaconf import OmegaConf
from dotenv import load_dotenv

load_dotenv()

config = OmegaConf.load(Path(os.environ["CONFIG_DIR"]) / "vae_config.yml")

dataset = get_dataset(config)

In [35]:
dataset[0].shape

torch.Size([10, 16, 16, 3])

In [23]:
from diffusers.models import AutoencoderKL

vae = AutoencoderKL(sample_size=16)


In [28]:
from einops import rearrange

x = dataset[0][0]
print(x.shape)
x = rearrange(x, 'b h w c -> b c h w')
print(x.shape)

vae.encode(x).latent_dist.mode()

torch.Size([99, 16, 16, 3])
torch.Size([99, 3, 16, 16])


tensor([[[[-2.4045e-01, -2.8419e-01, -2.4565e-01,  ..., -1.8972e-01,
           -2.6870e-01, -3.0393e-01],
          [-3.5516e-01, -5.6504e-01, -5.4501e-01,  ..., -3.8054e-01,
           -3.5006e-01, -2.9919e-01],
          [-3.7635e-01, -8.2154e-01, -7.4967e-01,  ..., -4.9417e-01,
           -4.5177e-01, -4.6670e-01],
          ...,
          [-3.7029e-01, -7.5063e-01, -4.7361e-01,  ..., -4.6213e-01,
           -5.3382e-01, -4.4760e-01],
          [-4.4875e-01, -8.3544e-01, -6.2338e-01,  ..., -5.5754e-01,
           -6.1721e-01, -4.9936e-01],
          [-3.4059e-01, -6.2864e-01, -5.4557e-01,  ..., -4.9456e-01,
           -5.2795e-01, -5.4594e-01]],

         [[ 1.8044e-01,  2.6652e-01,  2.8526e-01,  ...,  2.7473e-01,
            2.7866e-01,  3.7060e-01],
          [ 9.7396e-02,  1.7822e-01,  1.0779e-01,  ...,  1.4349e-01,
            2.6071e-01,  3.8604e-01],
          [ 9.8837e-02, -8.9740e-03, -1.0089e-01,  ...,  2.8887e-02,
            1.1255e-01,  2.4652e-01],
          ...,
     

In [3]:
from torchvision.io import write_video

input_norm = (dataset[69][0] - dataset[69][0].min()) / (dataset[69][0].max() - dataset[69][0].min()) * 255

write_video("test_video.mp4", input_norm, 10)

In [13]:
from PIL import Image
import numpy as np
from einops import rearrange

img = Image.fromarray(rearrange(np.array(input_norm[0]), "h w c -> c h w").astype(np.uint8).transpose(1, 2, 0))
img.save("test_image.png")

/tmp/ipykernel_44826/999753457.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  img = Image.fromarray(rearrange(np.array(input_norm[0]), "h w c -> c h w").astype(np.uint8).transpose(1, 2, 0))


In [10]:
input_norm[0].shape

torch.Size([16, 16, 3])

In [12]:
from timm.models.vision_transformer import VisionTransformer
import torch

model = VisionTransformer(img_size=16, patch_size=1, in_chans=3, num_classes=0, global_pool="", class_token=False, embed_dim=32, depth=2,
                          num_heads=1)

x = torch.randn(1, 3, 16, 16)

model(x).shape

torch.Size([1, 256, 32])

In [2]:
from vitssm.models import LNFPSimple
import torch

model = LNFPSimple()
x = torch.randn(1, 10, 16, 16, 3)

model(x).shape

torch.Size([10, 256, 32]) torch.Size([10, 256, 32]) torch.Size([10, 256, 32])
torch.Size([10, 256, 32]) torch.Size([10, 256, 32]) torch.Size([10, 256, 32])


torch.Size([1, 10, 16, 16, 3])

In [13]:
from omegaconf import OmegaConf, DictConfig
from pathlib import Path
import os
os.environ['CONFIG_DIR'] = os.getcwd() + "/configs"

base_config = OmegaConf.load(Path(os.environ['CONFIG_DIR']) / "base_config.yml")

def flatten_config(config, parent_key=None, sep='_'):
    items = []
    for k, v in config.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, DictConfig):
            items.extend(flatten_config(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return DictConfig(dict(items))

flattened_config = flatten_config(base_config)

flattened_config

{'experiment_project': 'ViTSSM', 'experiment_group': 'ViT', 'experiment_name': 'vmdsprites', 'experiment_log_freq': 25, 'dataset_name': 'vmdsprites', 'dataset_mode': 'train', 'dataset_fold': '0', 'dataset_resolution': 64, 'dataset_train_percentage': 0.9, 'dataset_num_workers': 8, 'dataset_pin_memory': True, 'optimization_loss_name': 'MSELoss', 'optimization_optimizer_name': 'AdamW', 'optimization_optimizer_kwargs_lr': 0.001, 'optimization_optimizer_kwargs_weight_decay': 0.01, 'optimization_scheduler_name': 'StepLR', 'optimization_scheduler_kwargs_step_size': 1, 'optimization_scheduler_kwargs_gamma': 0.5, 'optimization_epochs': 10, 'optimization_batch_size': 32, 'optimization_val_batch_size': 32, 'metrics': None, 'model_device': 'cuda', 'model_use_amp': True}

In [7]:
from vitssm.data.datasets import VideoMDSpritesDataset


dataset = VideoMDSpritesDataset("data_store/VMDsprites")

In [3]:
from vitssm.data import get_dataloaders, get_dataset
from vitssm.models import build_model
from omegaconf import OmegaConf
from pathlib import Path
import dotenv
import os
import torch

dotenv.load_dotenv()

base_config = OmegaConf.load(Path(os.environ["CONFIG_DIR"]) / "base_config.yml")

#dataset = get_dataset(base_config)
#
#model = build_model(base_config)
#
#checkpoint = torch.load("/home/david/projects/ViTSSM/checkpoints/ViTSSM/ViT/vmdsprites.pth")
#model.load_state_dict(checkpoint["model"])
#
#pred = model(dataset[69][0].unsqueeze(0))

dict(base_config)

{'project': 'ViTSSM',
 'group': 'ViT',
 'name': 'vmdsprites',
 'log_freq': 25,
 'seed': 420,
 'dataset': {'name': 'vmdsprites', 'mode': 'train', 'fold': '0', 'resolution': 64, 'train_percentage': 0.9, 'num_workers': 0, 'pin_memory': True, 'batch_size': 32, 'val_batch_size': 32},
 'optimization': {'loss': {'name': 'MSELoss', 'args': {}}, 'optimizer': {'name': 'AdamW', 'kwargs': {'lr': 0.001, 'weight_decay': 0.01}}, 'scheduler': {'name': 'StepLR', 'kwargs': {'step_size': 1, 'gamma': 0.5}}, 'epochs': 10},
 'metrics': {'MSE': {'name': 'MeanSquaredError', 'kwargs': {}}},
 'model': {'name': 'latent_nextframe_attention', 'device': 'cuda', 'use_amp': True, 'frame_in_size': [64, 64], 'latent_dim': 64, 'n_frames': 99}}

In [19]:
from torchvision.io import write_video

input_norm = (dataset[69][0] - dataset[69][0].min()) / (dataset[69][0].max() - dataset[69][0].min()) * 255
pred_norm = (pred - pred.min()) / (pred.max() - pred.min()) * 255

write_video("input.mp4", input_norm, 10)
write_video("pred.mp4", pred_norm[0], 10)

In [4]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchmetrics import MeanSquaredError

err = MeanSquaredError()

err(dataset[0][0], dataset[0][1])

tensor(0.9793)

In [6]:
from vitssm.models import LatentNextFramePrediction, LatentNextFramePredictionConfig

config = LatentNextFramePredictionConfig()
model = LatentNextFramePrediction(**config.model_dump())

In [4]:
from tqdm import tqdm

for _ in tqdm(dataloaders[0]):
    pass

100%|██████████| 226/226 [00:19<00:00, 11.48it/s]


In [13]:
from vitssm.models import LatentNextFramePrediction, LatentNextFramePredictionConfig

LatentNextFramePredictionConfig(**base_config.model)

LatentNextFramePredictionConfig(frame_in_size=(64, 64), frame_out_size=(224, 224), n_frames=30, window_size=10, patch_size=16, latent_dim=64, n_blocks_encoder=4, n_heads_encoder=4, n_blocks_latent_predictor=4, n_heads_latent_predictor=4, n_blocks_decoder=4, n_heads_decoder=4, mlp_multiplier=3, pos_enc_dropout=0.0, residual_dropout=0.0, mlp_dropout=0.0)

In [12]:
from torchvision.transforms.v2 import Compose, ToDtype
import torch

model = LatentNextFramePrediction(
    frame_in_size=(64, 64),
    frame_out_size=(64, 64),
    n_frames=99,
    patch_size=4,
    latent_dim=64,
    n_heads=2,
)

x = ToDtype(torch.float32, scale=True)(dataset[0][0].unsqueeze(0))
print(x.shape)
pred = model(x)
print(pred.shape)

torch.Size([1, 99, 64, 64, 3])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([99, 256, 64]) torch.Size([99, 256, 64]) torch.Size([99, 1, 64])
torch.Size([1, 99, 64, 64, 3])


In [1]:
import wandb

run = wandb.init(project="vitssm", entity="davidkuernsteiner")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: davidkuernsteiner. Use `wandb login --relogin` to force relogin


In [1]:
import torch
from torch import nn
from vitssm.models import LatentNextFramePrediction, LatentNextFramePredictionConfig



_x = torch.randn(1, 30, 64, 64, 3).cuda()
model = LatentNextFramePrediction().cuda()
with torch.autocast(device_type="cuda", dtype=torch.bfloat16, enabled=True):
    _pred = model(_x)
    
print(_pred.dtype)

/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


torch.Size([30, 256, 64]) torch.Size([30, 256, 64]) torch.Size([30, 1, 64])
torch.Size([30, 256, 64]) torch.Size([30, 256, 64]) torch.Size([30, 1, 64])
torch.Size([30, 256, 64]) torch.Size([30, 256, 64]) torch.Size([30, 1, 64])
torch.Size([30, 256, 64]) torch.Size([30, 256, 64]) torch.Size([30, 1, 64])
torch.bfloat16


/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
import wandb

wandb.init(config="/home/david/projects/ViTSSM/run_configs/base_config.yml")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TypeError: string indices must be integers, not 'str'

In [8]:
x, y = dataset[0]

In [3]:
import gdown

file_id = '1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV'
url = f'https://drive.google.com/uc?id={file_id}'
# Specify output location
output = 'VMDsprites.zip'  # Replace with desired file name

# Download the file
gdown.download(url, output, quiet=False)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV
From (redirected): https://drive.google.com/uc?id=1T7Vq7a70hu5949FJMe8yie5tvAwOmxXV&confirm=t&uuid=6a458c62-33fa-459d-a0c2-96a42732cbae
To: /home/david/projects/ViTSSM/VMDsprites.zip
100%|██████████| 272M/272M [00:07<00:00, 38.1MB/s] 


'VMDsprites.zip'

In [2]:
from vitssm.data.mdsprites.generators import generate_videos
from vitssm.data.mdsprites.shapes import Circle, Rectangle, Triangle

gen = iter(generate_videos(
    64,
    [Circle, Rectangle, Triangle],
    6,
    ["red", "blue", "green", "yellow", "purple"],
    "#000000",
    100,
))

for _ in range(10):
    print(next(gen)[1])

[[array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255,

In [6]:
from xformers.components.attention import ScaledDotProduct
from xformers.ops.fmha.attn_bias import LowerTriangularMask
import torch
from torch import nn

att = ScaledDotProduct(seq_len=30, to_seq_len=40)
q_proj = nn.Linear(128, 128)
k_proj = nn.Linear(128, 128)
v_proj = nn.Linear(128, 128)

q = torch.ones((32, 30, 128))
k = torch.ones((32, 30, 128))
v = torch.ones((32, 40, 128))
mask = LowerTriangularMask()

att(q_proj(q), k_proj(k), v_proj(v), mask=mask).shape

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 30] but got: [32, 40].

In [1]:
from vitssm.models.lnfp import LatentNextFramePrediction
import torch

model = LatentNextFramePrediction(frame_in_size=(32, 32), frame_out_size=(32, 32), patch_size=4, latent_dim=32, n_heads=2, n_frames=10)
x = torch.randn(32, 10, 32, 32, 3)
out = model(x)
print(out.shape)

/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE
/home/david/micromamba/envs/ViTSSM/lib/python3

torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([32, 10, 32, 32, 3])


In [3]:
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [1]:
from torchvision.datasets import MovingMNIST
from vitssm.data.datasets import NextFrameDataset


mnist = MovingMNIST(root="data", split="train", split_ratio=10, download=False)
data = NextFrameDataset(mnist)


(tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         ...,
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
      